In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pyclustertend import vat

## Loading Data and Randomly Sampling

In [ ]:
sp_data = pd.read_csv('special_traffic.csv', sep = ',', index_col = None)
# special_traffic = sp_data.sample(n = 1000, random_state = 123)
# special_traffic.head()

# 3.2(a)

## Extracted the x Value

In [ ]:
used_x = sp_data.loc[:,'maximum_speed': 'motorcycle']
used_x.head()

In [ ]:
y = sp_data[['StrType']]
y.head()

In [ ]:
y['StrType'].unique()

## PCA Plot with original x 

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(used_x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, y], axis = 1)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 Component PCA', fontsize = 20)


targets = ['Council Minor', 'Arterial']
colors = ['r', 'g']
markers = ['*', 'o']
for target, color, marker in zip(targets,colors, markers):
    indicesToKeep = finalDf['StrType'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1'],
               finalDf.loc[indicesToKeep, 'principal component 2'],
               c = color,
               marker = marker,
               s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
pca.explained_variance_ratio_

# 3.2(b)

In [ ]:
# From the explained variance ratio above, we know that the first two pricipal components has contained more than 99.7% of variance of
# the whole dataset. In addition, the first pricipal component contains 99.3% and the second contains 0.4%.
# From the plot above, we can easily separated data into two groups, Council Minor and Arterial, based on the two new features,
# the first Principal Conponent and the second Pricipal Conponent. We can condclude that variables that greater than -125 in first 
# Principal Conponent and greater than 50 in second Principal Conponent are Arterial. Otherwise, they are Council Minor.

# 3.2(c)

In [ ]:
vat(principalComponents)

In [ ]:
x = np.array(used_x)
vat(x)